In [187]:
import os
import numpy as np
import random
import csv
from tqdm import tqdm
import matplotlib.pyplot as plt
from dataset import dataset
from additional_functions import process_all

In [188]:
class neural_net:
    def __init__(self, data: dataset, prediction_type_flag: str, hidden_layer_count=0, momentum=1.0, learning_rate=1.0, batch_size=1, suppress_plots=True):
        self.suppress_plots = suppress_plots
        self.momentum = momentum
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        if hidden_layer_count == 0:
            hidden_node_count = 0
        else:
            hidden_node_count = [1] * hidden_layer_count
        self.tune_set = data.tune_set
        self.validate_set = data.validate_set
        self.prediction_type = prediction_type_flag

        if self.prediction_type == "classification":
            self.class_count = len(np.unique(self.tune_set[:,-1]))
        else:
            self.class_count = 0

        input_size = self.tune_set.shape[1] - 1
        self.network_shape = [input_size] + (hidden_node_count if hidden_node_count else []) + ([self.class_count] if (self.prediction_type == "classification") else [1])
        self.biases = []
        self.weights = []
        self.bias_velocity = []
        self.weight_velocity = []

    def init_weights_biases_momentum(self):
        '''
        Initializes weights based on the network shape list
        '''
        self.biases = [np.random.randn(next_size, 1) for next_size in self.network_shape[1:]]
        self.weights = [np.random.randn(next_size, cur_size) for cur_size, next_size in zip(self.network_shape[:-1], self.network_shape[1:])]
        self.bias_velocity = [np.zeros(bias.shape) for bias in self.biases]
        self.weight_velocity = [np.zeros(weight.shape) for weight in self.weights]
        #print(self.biases)


    def for_prop(self, input: np):
        '''
        Feeds forward a single example through the network
        '''
        output = input
        for bias, weight in zip(self.biases[:-1], self.weights[:-1]):
            output = self.sigmoid(np.dot(weight, output) + bias)
        # Not sure right now if output will be correct for regression, but life goes on.
        # The following lines choose the output activation function based on prediction type
        bias, weight = self.biases[-1], self.weights[-1]
        # MIGHT NEED TO RESHAPE THE DOT PRODUCT ON THE LINE BELOW
        output = (np.dot(weight, output) + bias)
        if self.prediction_type == "classification":
            output = self.softmax(output)
        return output
    
    def get_training_data(self, i: int):
        '''
        method needs to take the set of fold i-(i-1) and and compile those into its own array.
        Then format the data as follows: each example = (attributes, label)
        i is used to indicate which training set you want returned
        '''
        desired_data = np.concatenate([self.validate_set[j] for j in range(10) if j != i])
        training_data = [(example[:-1], example[-1]) for example in desired_data]
        return training_data
    
    def get_testing_data(self, i: int):
        '''
        method needs to take the set of fold i-(i-1) and and compile those into its own array.
        Then format the data as follows: each example = (attributes, label)
        i is used to indicate which training set you want returned
        '''
        desired_data = self.validate_set[i]
        testing_data = [(example[:-1], example[-1]) for example in desired_data]
        return testing_data

    def grad_desc(self, training_data, epochs, momentum, learning_rate, batch_size):
        '''
        Takes in a traing set from get_training_data. The format is a list of tuples, where each tuple
        represents an example. Within each tuple the first value is the feature vector and the second
        value is the label.
        '''
        example_count = len(training_data)
        for epoch in tqdm(range(epochs), desc="Training Epochs", leave=False):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+batch_size] for k in range(0, example_count, batch_size)]
            for mini_batch in mini_batches:
                self.update_weights(mini_batch, momentum, learning_rate)

    def update_weights(self, mini_batch, momentum, learning_rate):
        '''
        NEEDS COMMENTING/PARSING
        '''
        bias_gradient = [np.zeros(bias.shape) for bias in self.biases]
        weight_gradient = [np.zeros(weight.shape) for weight in self.weights]

        # Compute gradients for the mini-batch
        for feature, label in mini_batch:
            #print(type(label))
            if not np.isnan(label):
                delta_bias_gradient, delta_weight_gradient = self.epoch(feature, label)
                bias_gradient = [gradient + delta for gradient, delta in zip(bias_gradient, delta_bias_gradient)]
                weight_gradient = [gradient + delta for gradient, delta in zip(weight_gradient, delta_weight_gradient)]
        
        # Update velocities and apply updates with momentum
        self.bias_velocity = [momentum * velocity - (learning_rate / len(mini_batch)) * gradient for velocity, gradient in zip(self.bias_velocity, bias_gradient)]
        self.weight_velocity = [momentum * velocity - (learning_rate / len(mini_batch)) * gradient for velocity, gradient in zip(self.weight_velocity, weight_gradient)]

        # Update weights and biases
        self.biases = [bias + velocity for bias, velocity in zip(self.biases, self.bias_velocity)]
        self.weights = [bias + velocity for bias, velocity in zip(self.weights, self.weight_velocity)]

    def epoch(self, feature, label):
        '''
        NEEDS COMMENTING/PARSING
        '''
        #print(f"\n\nLABEL: {label}\n\n")
        bias_gradient = [np.zeros(bias.shape) for bias in self.biases]
        weight_gradient = [np.zeros(weight.shape) for weight in self.weights]
        # feedforward
        activation = feature
        activations = [feature] # list to store all the activations, layer by layer
        weighted_inputs = [] # list to store all the z vectors, layer by layer
        #print(f"Biases: {self.biases[-1]}")
        #print(f"Weights: {self.weights[-1]}")
        for bias, weight in zip(self.biases[:-1], self.weights[:-1]):
            weighted_input = np.dot(weight, activation) + bias
            activation = self.sigmoid(weighted_input)
            weighted_inputs.append(weighted_input)
            activations.append(activation)
        # The output layer uses different activation functions
        bias, weight = self.biases[-1], self.weights[-1]
        #print(f"Activation:\n{activation}\n\nWeight:\n{weight}\n\nBias:\n{bias.shape}\n\n\n")
        weighted_input = np.dot(weight, activation).reshape(-1,1) + bias
        #weighted_input = np.dot(weight, activation)
        #print(f"Weighted Input (Should be two scalars):\n{(weighted_input.shape)}")
        activation = weighted_input
        if self.prediction_type == "classification":
            activation = self.softmax(weighted_input)
        weighted_inputs.append(weighted_input)
        activations.append(activation)
        #print(f"Activations:\n{activations}\n\nWeighted Inputs:\n{weighted_inputs}")
    


        # backward pass
        # NEED TO ONE-HOT ENCODE THE LABELS FOR CLASSIFICATION SETS TO MAKE THE DELTA LINE WORK
        # PROLLY WON'T NEED LOSS PRIME METHOD
        if self.prediction_type == "classification":
            one_hot_label = [0] * self.class_count
            one_hot_label[int(label)] = 1
            one_hot_label = np.array(one_hot_label).reshape(-1, 1)
        else:
            one_hot_label = label
        #print(f"Activations[-1]:\n{activations[-1]}\nOne-Hot Label:\n{one_hot_label}")
        delta = (activations[-1] - one_hot_label)# * self.softmax(weighted_inputs[-1])
        bias_gradient[-1] = delta
        #print(f"Delta:\n{delta}\n\nActivations:\n{activations[-2]}\n\n")
        weight_gradient[-1] = np.dot(delta, activations[-2].reshape(1,-1))# # CHECK THIS LINE FOR COMPREHENSION

        for layer_idx in range(2, len(self.network_shape)):
            weighted_input = weighted_inputs[-layer_idx]
            activation_prime = self.sigmoid_prime(weighted_input)
            delta = np.dot(self.weights[-layer_idx+1].transpose(), delta) * activation_prime
            # add logic to convert scalar if delta is 1x1
            bias_gradient[-layer_idx] = delta
            weight_gradient[-layer_idx] = np.dot(delta, activations[-layer_idx-1].transpose())
        #print("GOT TO THE END")
        return (bias_gradient, weight_gradient)

    def tune(self):
        return
    def train(self):

        return
    def loss(self, test_data):
        if self.prediction_type == "classification":
            results = [(np.argmax(self.for_prop(example)), label) for (example, label) in test_data]
            return sum(int(example == label) for (example, label) in results)
        else:
            results = [(self.for_prop(example), label) for (example, label) in test_data]
            #answers = np.array(results[0], dtype=float)
            #labels = np.array(results[1], dtype=float)
            #mse = np.mean(answers - labels) ** 2
            return np.mean(np.array(results[0], dtype=float) - np.array(results[1], dtype=float)) ** 2
    '''
    def loss_prime(self):
        return
    '''
    def sigmoid(self, input: np):
        return 1.0/(1.0+np.exp(-input))
    def sigmoid_prime(self, input: np):
        return self.sigmoid(input)*(1-self.sigmoid(input))
    def softmax(self, input):
        exp = np.exp(input - np.max(input))
        return exp / np.sum(exp)
    '''
    # Since loss output is in a slightly different format for neural nets, we might need an final loss method to output final performance
    def final_loss(self):
        return
    '''
    # THIS PROLLY NEEDS EDITING
    def plot_loss(self, metrics: list, parameter: str, increment):
        '''
        This function plots the loss performance for each epoch. This allows us to visualize at how many epochs
        performance drops off.
        '''
        # Extract # of epochs and loss metrics
        metrics = np.array(metrics)
        epochs = np.arange(1, metrics.shape[0] + 1) * increment
        loss1 = metrics[:, 0]
        loss2 = metrics[:, 1]

        # Create loss plot
        plt.figure(figsize=(10, 6))
        plt.plot(epochs, loss1, label='Loss Metric 1', marker='o')
        plt.plot(epochs, loss2, label='Loss Metric 2', marker='o')
        plt.xlabel(f'{parameter} Value')
        plt.ylabel('Loss')
        plt.title(f'Loss Metrics vs. {parameter} value')
        plt.legend()
        plt.grid(True)
        plt.show()
        plt.close()
    

In [189]:
abalone_data, cancer_data, fire_data, glass_data, machine_data, soybean_data = process_all('carlthedog3', True)

In [190]:
num_hidden_layers = 0
cancer_net = neural_net(cancer_data, "classification", hidden_layer_count=num_hidden_layers)
print(cancer_net.network_shape)
cancer_net.init_weights_biases_momentum()
cancer_net.grad_desc(cancer_net.get_training_data(0), 100, .9, .05, 5)
score = cancer_net.loss(cancer_net.get_testing_data(0))
#print(score)
print(score/(37))

[9, 2]


0.5675675675675675


In [191]:
num_hidden_layers = 1
fire_net = neural_net(fire_data, "regression", hidden_layer_count=num_hidden_layers)
print(fire_net.network_shape)
fire_net.init_weights_biases_momentum()
fire_net.grad_desc(fire_net.get_training_data(0), 10, .9, .05, 5)
fire_net.loss(fire_net.get_testing_data(0))

[12, 1, 1]


ValueError: shapes (1,1) and (12,) not aligned: 1 (dim 1) != 12 (dim 0)

In [65]:
'''
Weight initialization verification
'''

'''
num_hidden_layers = 2
cancer_net = neural_net(cancer_data, "classification", hidden_layer_count=num_hidden_layers, hidden_node_count=3)
x = cancer_net.init_weights(5)
for i in range(num_hidden_layers+1):
    print(x[i].shape)

print("\n")

num_hidden_layers = 1
fire_net = neural_net(fire_data, "regression", hidden_layer_count=num_hidden_layers, hidden_node_count=3)
x = fire_net.init_weights(3)
for i in range(num_hidden_layers+1):
    print(x[i].shape)
'''

'\nnum_hidden_layers = 2\ncancer_net = neural_net(cancer_data, "classification", hidden_layer_count=num_hidden_layers, hidden_node_count=3)\nx = cancer_net.init_weights(5)\nfor i in range(num_hidden_layers+1):\n    print(x[i].shape)\n\nprint("\n")\n\nnum_hidden_layers = 1\nfire_net = neural_net(fire_data, "regression", hidden_layer_count=num_hidden_layers, hidden_node_count=3)\nx = fire_net.init_weights(3)\nfor i in range(num_hidden_layers+1):\n    print(x[i].shape)\n'